# 07. Pruning Study

This notebook explores network pruning techniques to reduce model size.

## Experiment Overview
- **Goal**: Explore network pruning techniques
- **Model**: Prunable MLP with magnitude-based pruning
- **Features**: Pruning schedules, sparsity analysis, accuracy vs. compression
- **Learning**: Understanding model compression and efficiency

## What You'll Learn
- Network pruning techniques
- Magnitude-based pruning
- Pruning schedules
- Accuracy vs. compression trade-offs


In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

# Add scripts directory to path
sys.path.append('../scripts')
from utils import load_mnist_data, get_device, set_seed

# Set random seed for reproducibility
set_seed(42)

# Get device
device = get_device()
print(f"Using device: {device}")

# Load MNIST dataset
print("Loading MNIST dataset...")
train_loader, val_loader, test_loader = load_mnist_data(batch_size=64, test_split=0.2)

print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")
